# このコードの目的

少し複雑なネットワークをtensor.nnを使って簡単に定義する方法をまとめる。

In [11]:
import torch
import random

In [46]:
N = 1000
Dim_input = 5
Dim_output = 1

# toy dataを作成する。
# このデータは単純な線形結合では不可能でネットワークに工夫が必要
x = torch.rand((N, Dim_input))
y = 2 * x[:, 0] + 10 * x[:, 1] * x[:, 2]

x_train, y_train = x[:900], y[:900]
x_test, y_test = x[900:], y[900:]


In [47]:
# シンプルな解決法として、適切なactivation関数を設定した多層ネットワークを定義する
model = torch.nn.Sequential(
            torch.nn.Linear(Dim_input,  10, bias=None),
            torch.nn.ReLU(),
            torch.nn.Linear(10,  10, bias=None),
            torch.nn.ReLU(),
            torch.nn.Linear(10,  Dim_output, bias=None),
            )
criterion = torch.nn.L1Loss(reduction='mean') 
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=0.01)

# 精度を最大化するために、batch_size=1で学習する。
batch_size = 1
epoch_size = 1000
for t in range(epoch_size):
    batch_indexes = random.choices(range(x_train.shape[0]), k=batch_size)
    _x, _y = x_train[batch_indexes], y_train[batch_indexes]
    loss = criterion(model(_x), _y)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
# test_setで学習結果を確認する    
for predict, answer in zip(model(x_test), y_test):
    print(predict, answer)

# ある程度は予測ができている。

tensor([0.8912], grad_fn=<SelectBackward>) tensor(0.8997)
tensor([6.3331], grad_fn=<SelectBackward>) tensor(5.8027)
tensor([4.8072], grad_fn=<SelectBackward>) tensor(6.6585)
tensor([2.7111], grad_fn=<SelectBackward>) tensor(2.7360)
tensor([1.0591], grad_fn=<SelectBackward>) tensor(0.8989)
tensor([4.9784], grad_fn=<SelectBackward>) tensor(6.1582)
tensor([5.5727], grad_fn=<SelectBackward>) tensor(5.9645)
tensor([3.5766], grad_fn=<SelectBackward>) tensor(2.5390)
tensor([5.2392], grad_fn=<SelectBackward>) tensor(4.4531)
tensor([2.4924], grad_fn=<SelectBackward>) tensor(2.3779)
tensor([3.0132], grad_fn=<SelectBackward>) tensor(1.4361)
tensor([6.4035], grad_fn=<SelectBackward>) tensor(8.0534)
tensor([2.2899], grad_fn=<SelectBackward>) tensor(1.5370)
tensor([6.7072], grad_fn=<SelectBackward>) tensor(8.3103)
tensor([4.6894], grad_fn=<SelectBackward>) tensor(3.5375)
tensor([2.2125], grad_fn=<SelectBackward>) tensor(2.0919)
tensor([3.0127], grad_fn=<SelectBackward>) tensor(1.0422)
tensor([0.3743

In [49]:
# toy data の生成モデルに等しいネットワーク構成になるようにmodelを組み合わせてみる。

simple_linear_model = torch.nn.Sequential(
            torch.nn.Linear(Dim_input,  Dim_output, bias=None),
            )
linear_cross_section_model = torch.nn.Sequential(
            torch.nn.Linear(Dim_input,  1, bias=None),
            )

criterion = torch.nn.L1Loss(reduction='mean') 
optimizer = torch.optim.Adam(
                            [
                                {'params':simple_linear_model.parameters()}, 
                                {'params':linear_cross_section_model.parameters()}
                            ]
                             , lr=0.01, weight_decay=0.01
            )

# 精度を最大化するために、batch_size=1で学習する。
batch_size = 1
epoch_size = 1000
for t in range(epoch_size):
    batch_indexes = random.choices(range(x_train.shape[0]), k=batch_size)
    _x, _y = x_train[batch_indexes], y_train[batch_indexes]
    _y_pre = simple_linear_model(_x) + linear_cross_section_model(_x)
    loss = criterion(_y_pre, _y)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
# test_setで学習結果を確認する    
for _x, answer in zip(x_test, y_test):
    predict = simple_linear_model(_x) + linear_cross_section_model(_x)
    print(predict, answer)

# ある程度は予測ができている。

tensor([0.6429], grad_fn=<AddBackward0>) tensor(0.8997)
tensor([5.7105], grad_fn=<AddBackward0>) tensor(5.8027)
tensor([4.8730], grad_fn=<AddBackward0>) tensor(6.6585)
tensor([2.8755], grad_fn=<AddBackward0>) tensor(2.7360)
tensor([0.9719], grad_fn=<AddBackward0>) tensor(0.8989)
tensor([4.8311], grad_fn=<AddBackward0>) tensor(6.1582)
tensor([5.1129], grad_fn=<AddBackward0>) tensor(5.9645)
tensor([3.2644], grad_fn=<AddBackward0>) tensor(2.5390)
tensor([4.6793], grad_fn=<AddBackward0>) tensor(4.4531)
tensor([2.4203], grad_fn=<AddBackward0>) tensor(2.3779)
tensor([2.7729], grad_fn=<AddBackward0>) tensor(1.4361)
tensor([6.2935], grad_fn=<AddBackward0>) tensor(8.0534)
tensor([2.3044], grad_fn=<AddBackward0>) tensor(1.5370)
tensor([6.3652], grad_fn=<AddBackward0>) tensor(8.3103)
tensor([4.3283], grad_fn=<AddBackward0>) tensor(3.5375)
tensor([1.8659], grad_fn=<AddBackward0>) tensor(2.0919)
tensor([3.1242], grad_fn=<AddBackward0>) tensor(1.0422)
tensor([0.1976], grad_fn=<AddBackward0>) tensor(